# Data exploration

In this notebook we explore the raw data consisting on 4 excel files: 
- Preinscripció.xlsl
- Matricules.xlsl
- Notes.xlsl
- Titulacions.xlsl

In [1]:
import pandas as pd
import re
from unidecode import unidecode

## Preinscripció

In [2]:
file_path = '../data/raw/Preinscripció.xlsx'
preinscripcions_df = pd.read_excel(file_path)

In [3]:
preinscripcions_df.shape

(2280, 77)

In [4]:
preinscripcions_df.sample()

,Llista,Curs,Convocatoria,DNI,CodiCentreSecundaria,Nacionalitat,Pais,CP,Poblacio,Comarca,...,FrancesCentreExtern,AltresIdiomes,AltresComprensioEscrita,AltresComprensioOral,AltresExpresioEscrita,AltresExpresioOral,AltresBatxillerat,AltresCentreExtern,NoIdiomesCentreExtern,OrdinadorDomicili
4,J02,2014,J,9662817A,8024297.0,ESP,ESP,8208,Sabadell,40.0,...,N,NaN,NaN,NaN,NaN,NaN,N,N,S,1.0


In [5]:
preinscripcions_df.columns

Index(['Llista', 'Curs', 'Convocatoria', 'DNI', 'CodiCentreSecundaria',
       'Nacionalitat', 'Pais', 'CP', 'Poblacio', 'Comarca', 'Provincia',
       'AutoritzacioEnvInfAccesUniv', 'Sexe', 'DataNaixement',
       'UniversitatProcedenciaPAU', 'ViaAcces', 'OpcioCOU', 'NotaAcces',
       'Pref_1', 'Pref_2', 'Pref_3', 'Pref_4', 'Pref_5', 'Pref_6', 'Pref_7',
       'Pref_8', 'CentreAsignat', 'OrdreAssignacio', 'JustificantTrasllat',
       'AnyQualificacio', 'ConvocatoriaQualificacio', 'Concordancia',
       'EstudisPare', 'EstudisMare', 'OcupacioPare', 'OcupacioMare',
       'TreballRemunerat', 'Orientacio', 'AspectesEleccio', 'Altres',
       'AnyAcabamentSecundaria', 'TipusCentreSecundaria',
       'TipusLlocSecundaria', 'CodiLlocSecundaria', 'EstudisUniversitaris',
       'AnyAccesPrimeraVegadaUniversitat', 'TitulacioNivellAssolit',
       'ModalitatAcces', 'CatalaComprensioEscrita', 'CatalaComprensioOral',
       'CatalaExpresioEscrita', 'CatalaExpresioOral',
       'CastellaComprens

In [6]:
#Dropping variables because of irrelevancy: Llista, AutoritzacioEnvInfAccesUniv, UniversitatProcedenciaPAU, 
        #Prefs (Keeping OrdreAssignacio)
#Dropping variables because of Naans: OpcioCOU, JustificantTrasllat, Altres, OrdinadorDomicili

columns_to_drop = ['Llista', 'AutoritzacioEnvInfAccesUniv', 'UniversitatProcedenciaPAU', 'OpcioCOU',
                   'Pref_1','Pref_2', 'Pref_3', 'Pref_4', 'Pref_5', 'Pref_6', 'Pref_7', 'Pref_8',
                    'JustificantTrasllat','Concordancia', 'Altres', 'OrdinadorDomicili']

preinscripcions_df = preinscripcions_df.drop(columns=columns_to_drop)

In [7]:
renamed_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in preinscripcions_df.columns}
preinscripcions_df = preinscripcions_df.rename(columns=renamed_columns)

In [8]:
#'Centreassignat' is the degree (grau) that each student has been assigned to
# Curs is the year they appear on the preinscription list for that specific degree
# Changing the name for better understanding

preinscripcions_df.rename(columns={'curs':'curs_preinscripcio',
                                   'd_n_i':'DNI',
                                   'c_p':'CP',
                                   'centre_asignat': 'codi_grau'}, inplace=True)


In [9]:
#We are going to remove accents from all the df to avoid errors in SQL. We use unidecode library.
def remove_accents(text):
    return unidecode(text)

In [10]:
preinscripcions_df['DNI'] = preinscripcions_df['DNI'].apply(remove_accents)
preinscripcions_df['poblacio'] = preinscripcions_df['poblacio'].apply(remove_accents)

In [72]:
preinscripcions_df = preinscripcions_df.dropna(subset=['convocatoria_qualificacio'])

In [71]:
preinscripcions_df['nota_acces'] = preinscripcions_df['nota_acces']/1000

In [11]:
preinscripcions_df.shape

(2280, 61)

We are going to divide the columns of preinscriptions_df in 4 to make it more manageable. 
From one unique df we will have:
- Student variables
- University access variables
- Previous education variables
- Language variables

#### Students information

In [12]:
students_df = preinscripcions_df.copy()
columns_to_drop = ['curs_preinscripcio', 'convocatoria', 'codi_centre_secundaria',
        'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
        'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces',
       'catala_comprensio_escrita', 'catala_comprensio_oral',
       'catala_expresio_escrita', 'catala_expresio_oral',
       'castella_comprensio_escrita', 'castella_comprensio_oral',
       'castella_expresio_escrita', 'castella_expresio_oral',
       'angles_comprensio_escrita', 'angles_comprensio_oral',
       'angles_expresio_escrita', 'angles_expresio_oral', 'angles_batxillerat',
       'angles_centre_extern', 'frances_comprensio_escrita',
       'frances_comprensio_oral', 'frances_expresio_escrita',
       'frances_expresio_oral', 'frances_batxillerat', 'frances_centre_extern',
       'altres_idiomes', 'altres_comprensio_escrita', 'altres_comprensio_oral',
       'altres_expresio_escrita', 'altres_expresio_oral', 'altres_batxillerat',
       'altres_centre_extern', 'no_idiomes_centre_extern']

students_df = students_df.drop(columns=columns_to_drop)

In [13]:
#We want DNI to be unique to later use it as primary key
students_df = students_df.groupby('DNI').first().reset_index()

In [14]:
print(f' Duplicates:{students_df.duplicated().sum()}')
print('--------------')
print('Null values:')
print(students_df.isnull().sum())

 Duplicates:0
--------------
Null values:
DNI                    0
nacionalitat           0
pais                   0
CP                     0
poblacio               0
comarca              158
provincia             34
sexe                   0
data_naixement         0
estudis_pare           0
estudis_mare           0
ocupacio_pare          0
ocupacio_mare          0
treball_remunerat      0
dtype: int64


#### University access variables

In [15]:
uni_access_df = preinscripcions_df.copy()
columns_to_drop = ['codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces',
       'catala_comprensio_escrita', 'catala_comprensio_oral',
       'catala_expresio_escrita', 'catala_expresio_oral',
       'castella_comprensio_escrita', 'castella_comprensio_oral',
       'castella_expresio_escrita', 'castella_expresio_oral',
       'angles_comprensio_escrita', 'angles_comprensio_oral',
       'angles_expresio_escrita', 'angles_expresio_oral', 'angles_batxillerat',
       'angles_centre_extern', 'frances_comprensio_escrita',
       'frances_comprensio_oral', 'frances_expresio_escrita',
       'frances_expresio_oral', 'frances_batxillerat', 'frances_centre_extern',
       'altres_idiomes', 'altres_comprensio_escrita', 'altres_comprensio_oral',
       'altres_expresio_escrita', 'altres_expresio_oral', 'altres_batxillerat',
       'altres_centre_extern', 'no_idiomes_centre_extern']

uni_access_df = uni_access_df.drop(columns=columns_to_drop)

In [16]:
uni_access_df.columns

Index(['curs_preinscripcio', 'convocatoria', 'DNI', 'via_acces', 'nota_acces',
       'codi_grau', 'ordre_assignacio', 'any_qualificacio',
       'convocatoria_qualificacio', 'orientacio', 'aspectes_eleccio'],
      dtype='object')

In [17]:
column_order = ['DNI', 'curs_preinscripcio', 'convocatoria', 'via_acces', 'nota_acces',
       'codi_grau', 'ordre_assignacio', 'any_qualificacio',
       'convocatoria_qualificacio', 'orientacio', 'aspectes_eleccio']
uni_access_df = uni_access_df[column_order]

In [18]:
print(f' Duplicates:{uni_access_df.duplicated().sum()}')
print('--------------')
print('Null values:')
print(uni_access_df.isnull().sum())

 Duplicates:0
--------------
Null values:
DNI                           0
curs_preinscripcio            0
convocatoria                  0
via_acces                     0
nota_acces                    0
codi_grau                     0
ordre_assignacio              0
any_qualificacio              0
convocatoria_qualificacio    14
orientacio                    0
aspectes_eleccio              0
dtype: int64


In [20]:
uni_access_df['nota_acces'] = uni_access_df['nota_acces']/1000

In [21]:
uni_access_df['nota_acces']

0       6.440
1       8.546
2       6.472
3       6.432
4       5.428
        ...  
2275    7.335
2276    7.054
2277    6.662
2278    7.140
2279    7.140
Name: nota_acces, Length: 2266, dtype: float64

#### Previous education information

In [22]:
previous_education_df = preinscripcions_df.copy()
columns_to_drop = ['curs_preinscripcio', 'convocatoria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'catala_comprensio_escrita', 'catala_comprensio_oral',
       'catala_expresio_escrita', 'catala_expresio_oral',
       'castella_comprensio_escrita', 'castella_comprensio_oral',
       'castella_expresio_escrita', 'castella_expresio_oral',
       'angles_comprensio_escrita', 'angles_comprensio_oral',
       'angles_expresio_escrita', 'angles_expresio_oral', 'angles_batxillerat',
       'angles_centre_extern', 'frances_comprensio_escrita',
       'frances_comprensio_oral', 'frances_expresio_escrita',
       'frances_expresio_oral', 'frances_batxillerat', 'frances_centre_extern',
       'altres_idiomes', 'altres_comprensio_escrita', 'altres_comprensio_oral',
       'altres_expresio_escrita', 'altres_expresio_oral', 'altres_batxillerat',
       'altres_centre_extern', 'no_idiomes_centre_extern']

previous_education_df = previous_education_df.drop(columns=columns_to_drop)

In [23]:
print(f' Duplicates:{previous_education_df.duplicated().sum()}')
print('--------------')
print('Null values:')
print(previous_education_df.isnull().sum())

 Duplicates:43
--------------
Null values:
DNI                                        0
codi_centre_secundaria                   776
any_acabament_secundaria                 399
tipus_centre_secundaria                  399
tipus_lloc_secundaria                    399
codi_lloc_secundaria                     401
estudis_universitaris                      0
any_acces_primera_vegada_universitat    1729
titulacio_nivell_assolit                1729
modalitat_acces                         1729
dtype: int64


#### Languages variables

In [24]:
languages_df = preinscripcions_df.copy()
columns_to_drop = ['curs_preinscripcio', 'convocatoria', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces']

languages_df = languages_df.drop(columns=columns_to_drop)

Analysis of languages_df to understand the data better:

In [25]:
print(f' Duplicates:{languages_df.duplicated().sum()}')
print('--------------')
print('Null values:')
print(languages_df.isnull().sum())

 Duplicates:96
--------------
Null values:
DNI                               0
catala_comprensio_escrita         0
catala_comprensio_oral            0
catala_expresio_escrita           2
catala_expresio_oral              2
castella_comprensio_escrita       0
castella_comprensio_oral       1995
castella_expresio_escrita         2
castella_expresio_oral         2074
angles_comprensio_escrita      2074
angles_comprensio_oral         2074
angles_expresio_escrita        2074
angles_expresio_oral           2074
angles_batxillerat             2074
angles_centre_extern           2074
frances_comprensio_escrita     2074
frances_comprensio_oral        2074
frances_expresio_escrita       2074
frances_expresio_oral          2074
frances_batxillerat            2074
frances_centre_extern          2074
altres_idiomes                 2254
altres_comprensio_escrita      2202
altres_comprensio_oral         2207
altres_expresio_escrita        2208
altres_expresio_oral           2206
altres_batxillerat   

In [26]:
def see_value_counts(column):
    value_counts = languages_df[column].value_counts()
    print(f'{column} has this {value_counts}')

In [27]:
for column in languages_df:
    see_value_counts(column)
#Languages information doesn't seem to be reliable, 
#there are many naan values and the columns that have values do not make sense 
#e.g. Number of N in catalan_expresion_escrita = 2067 but most students come from catalan schools

DNI has this DNI
8579042I    3
5076237D    3
1910908I    3
4548897O    3
3380393E    3
           ..
3692229D    1
4629923L    1
3350877U    1
3208908A    1
5031499I    1
Name: count, Length: 2154, dtype: int64
catala_comprensio_escrita has this catala_comprensio_escrita
N    1378
S     701
B     189
R      12
Name: count, dtype: int64
catala_comprensio_oral has this catala_comprensio_oral
N    2014
B     187
S      65
R      14
Name: count, dtype: int64
catala_expresio_escrita has this catala_expresio_escrita
N    2067
B     158
R      35
S      18
Name: count, dtype: int64
catala_expresio_oral has this catala_expresio_oral
N    2074
B     175
R      21
S       8
Name: count, dtype: int64
castella_comprensio_escrita has this castella_comprensio_escrita
N    2009
B     201
S      65
R       5
Name: count, dtype: int64
castella_comprensio_oral has this castella_comprensio_oral
B    202
1     33
3     19
7     15
5      6
4      5
R      4
6      1
Name: count, dtype: int64
castella_expr

In [28]:
#We take the decision not to include languages in the analysis

In [29]:
columns_to_drop = ['catala_comprensio_escrita', 'catala_comprensio_oral',
       'catala_expresio_escrita', 'catala_expresio_oral',
       'castella_comprensio_escrita', 'castella_comprensio_oral',
       'castella_expresio_escrita', 'castella_expresio_oral',
       'angles_comprensio_escrita', 'angles_comprensio_oral',
       'angles_expresio_escrita', 'angles_expresio_oral', 'angles_batxillerat',
       'angles_centre_extern', 'frances_comprensio_escrita',
       'frances_comprensio_oral', 'frances_expresio_escrita',
       'frances_expresio_oral', 'frances_batxillerat', 'frances_centre_extern',
       'altres_idiomes', 'altres_comprensio_escrita', 'altres_comprensio_oral',
       'altres_expresio_escrita', 'altres_expresio_oral', 'altres_batxillerat',
       'altres_centre_extern', 'no_idiomes_centre_extern']

preinscripcions_df = preinscripcions_df.drop(columns=columns_to_drop)

In [30]:
preinscripcions_df.columns

Index(['curs_preinscripcio', 'convocatoria', 'DNI', 'codi_centre_secundaria',
       'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca', 'provincia',
       'sexe', 'data_naixement', 'via_acces', 'nota_acces', 'codi_grau',
       'ordre_assignacio', 'any_qualificacio', 'convocatoria_qualificacio',
       'estudis_pare', 'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'treball_remunerat', 'orientacio', 'aspectes_eleccio',
       'any_acabament_secundaria', 'tipus_centre_secundaria',
       'tipus_lloc_secundaria', 'codi_lloc_secundaria',
       'estudis_universitaris', 'any_acces_primera_vegada_universitat',
       'titulacio_nivell_assolit', 'modalitat_acces'],
      dtype='object')

## Titulacions

In [31]:
file_path = '../data/raw/Titulacions.xlsx'
titulacions_df = pd.read_excel(file_path)

In [32]:
titulacions_df = titulacions_df.drop(columns='Estudi')

In [33]:
titulacions_df.columns = titulacions_df.columns.str.replace(' ', '_').str.lower()

In [34]:
titulacions_df.rename(columns={'preinscripció':'codi_grau'}, inplace=True)

In [35]:
#We will delete those rows with Naan values on grau because we are only interested in keeping university degrees
#(not masters, exchanges...)
titulacions_df = titulacions_df.dropna(subset=['codi_grau'])

In [36]:
titulacions_df = titulacions_df.astype({'codi_grau':'int'})

## Matricules

In [37]:
file_path = '../data/raw/Matrícules.xlsx'
matricules_df = pd.read_excel(file_path)

In [38]:
matricules_df.shape

(7679, 5)

In [39]:
matricules_df = matricules_df.drop(columns='NIP')
matricules_df.rename(columns={'Curs':'curs_matricula',
                             'Pla':'pla'}, inplace=True)

In [40]:
#We need to add a column in matriculations that will serve as link with the other dfs. This will be "codi_grau". 
#We will use the titulations df as guide to assign the corresponding "codi_grau to each row"
titulacions_df

,pla,descripció_pla_estudis,codi_grau
0,1467,Doble Grau en Enginyeria Electrònica Industria...,21112
1,1278,Doble Grau en Enginyeria Mecànica / Enginyeria...,21112
7,877,Grau en Enginyeria Electrònica Industrial i Au...,21025
8,1464,Grau en Enginyeria Electrònica Industrial i Au...,21025
9,878,Grau en Enginyeria Elèctrica (pla 2009),21023
10,879,Grau en Enginyeria Mecànica (pla 2009),21026
11,1462,Grau en Enginyeria Mecànica (pla 2020),21026
12,1450,Grau en Enginyeria d'Automoció (pla 2020),21132
13,1430,Grau en Enginyeria en Energies Renovables i Ef...,21127
14,880,Grau en Enginyeria en Organització Industrial ...,21022


In [41]:
#One same student could be twice in preinscription list for different degrees, 
#preinscripcions_df['DNI'].is_unique == False
    
    #If matriculated, this would be reflected as two rows in the matriculation df with same DNI but different codi_grau
    #However, in matricules_df we do not have 'codi_grau', we only have 'pla' so we will first use the information on the 
        #titulacions_df to create the codi_grau
        
def assign_codi_grau(row):
    if row['pla'] in [1467, 1278]:
        return 21112
    elif row['pla'] in [877, 1464]:
        return 21025
    elif row['pla'] == 878:
        return 21023
    elif row['pla'] in [879, 1462]:
        return 21026
    elif row['pla'] == 1450:
        return 21132
    elif row['pla'] == 1430:
        return 21127
    elif row['pla'] in [880, 1466]:
        return 21022
    else:
        return np.nan

matricules_df['codi_grau'] = matricules_df.apply(assign_codi_grau, axis=1)

In [42]:
matricules_df['DNI'].value_counts()

DNI
1959497A    24
6041427E    22
5541571A    21
0195308N    21
8390059A    21
            ..
4420667A     1
4379223L     1
4377297A     1
4366897E     1
HCCLPJRR     1
Name: count, Length: 2252, dtype: int64

In [43]:
#Similar to preinscriptions, we are going to remove accents to prevent future issues
matricules_df['DNI'] = matricules_df['DNI'].apply(remove_accents)

In [44]:
# Now we want to keep only the matriculations for the 1st year of a student. 
# Some students (DNI) matriculate in differrent degrees so we create an identification column aggregating DNI + codi_grau
    #so we do not delete 1st years of different degrees. 
matricules_df['identification'] = matricules_df[['DNI', 'codi_grau']].astype({'DNI': str, 'codi_grau': str}).agg(' / '.join, axis=1)
matricules_df = matricules_df.sort_values(by='curs_matricula')
matricules_df = matricules_df.groupby('identification').first().reset_index()

In [45]:
matricules_df['curs_matricula'].value_counts().sort_index()

curs_matricula
2013    639
2014    194
2015    156
2016    197
2017    198
2018    148
2019    171
2020    147
2021    218
2022    236
2023    244
Name: count, dtype: int64

In [46]:
#Since we have data from 2013 that year is adding all uni students matriculated that year (1st appearance of their DNI).
#We will drop students with curs_matricula = 2013 so we can properly work with data starting at 2014
matricules_df = matricules_df[matricules_df['curs_matricula'] > 2013]

In [47]:
matricules_df['DNI'].value_counts().head(20)

DNI
7852608I    4
6041427E    4
3340018A    3
7252929O    3
7339746A    3
4548897O    3
1807318G    3
0482351D    3
8020201I    3
3462297A    3
6358946E    3
7254365E    3
4352628D    3
1959497A    3
8445987I    3
3491189R    3
7236871R    3
8390059A    3
2468635A    3
9625183A    3
Name: count, dtype: int64

In [48]:
matricules_df[matricules_df['DNI'] == '3340018A']

,identification,curs_matricula,DNI,NIA,pla,codi_grau
813,3340018A / 21025,2016,3340018A,1459442,877,21025
814,3340018A / 21026,2016,3340018A,1459442,879,21026
815,3340018A / 21112,2016,3340018A,1459442,1278,21112


In [49]:
#We observe that many of the 3 times repeated DNIs is from students of the double degree
#This students have matriculations of 3 degrees on the same year, 
    #we delete the duplicates keeping just the codi_grau of the double degree (21112)

In [50]:
dni_counts_dict = matricules_df['DNI'].value_counts().to_dict()
dni_list_3_counts = [DNI for DNI, count in dni_counts_dict.items() if count == 3]
for dni in dni_list_3_counts:
    matriculations = matricules_df[matricules_df['DNI'] == dni]
    for index, row in matriculations.iterrows():
        codi_graus = list(matriculations.codi_grau)
        if 21112 in codi_graus and len(set(matriculations['curs_matricula'])) == 1:
            rows_to_drop = matriculations[(matriculations['codi_grau'] == 21025) | (matriculations['codi_grau'] == 21026)].index
            for i in rows_to_drop.values:
                matricules_df = matricules_df.drop(matricules_df[matricules_df.index == i].index)

In [51]:
matricules_df.shape

(1845, 6)

In [52]:
matricules_df[matricules_df['DNI'] == '3340018A']

,identification,curs_matricula,DNI,NIA,pla,codi_grau
815,3340018A / 21112,2016,3340018A,1459442,1278,21112


In [53]:
#The DNIs that are still duplicated are probable from people that has changed degrees in different years.
#We are going to keep just the first year they matriculated for this analysis taking as a rule
#that we want to study how was the first contact from students with the university

In [54]:
repeated_dni_list = [DNI for DNI, count in dni_counts_dict.items() if count >= 2]
for dni in repeated_dni_list:
    matriculations = matricules_df[matricules_df['DNI'] == dni]
    for index, row in matriculations.iterrows():
        if len(set(matriculations['curs_matricula'])) >= 2:
            rows_to_drop = matriculations[(matriculations['curs_matricula'] != matriculations['curs_matricula'].min())].index
            for i in rows_to_drop.values:
                matricules_df = matricules_df.drop(matricules_df[matricules_df.index == i].index)

In [55]:
matricules_df.shape

(1713, 6)

In [56]:
#There are some DNIs that are still repeated because students enrolled in multiple degrees the same year.
#We will manage this cases later in the analysis

## Notes

In [57]:
file_path = '../data/raw/Notes.xlsx'
notes_df = pd.read_excel(file_path)

In [58]:
notes_df.shape

(41327, 15)

In [59]:
notes_df.sample()

,ANO_ACADEMICO,ASSIG,CODIASS,GRUP,NIA,NIP,PLA,EXTRA,TIPUS,CURS,SEM,EXHAURIDES,SUPERADA,NOTANUMERICAACTA,NOTAALFAACTA
8247,2015,Mates,200268,101,1421525,422495,880,N,B,1,1,1,S,6.3,AP


In [60]:
notes_df = notes_df.drop(columns='EXTRA')
notes_df = notes_df.drop(columns='NIP')

In [61]:
notes_df.columns = notes_df.columns.map(lambda x: x.lower())

In [62]:
notes_df.columns

Index(['ano_academico', 'assig', 'codiass', 'grup', 'nia', 'pla', 'tipus',
       'curs', 'sem', 'exhaurides', 'superada', 'notanumericaacta',
       'notaalfaacta'],
      dtype='object')

In [63]:
notes_df.rename(columns={'ano_academico':'any_academic',
                             'assig':'assignatura',
                             'codiass':'codi_assignatura',
                             'nia': 'NIA',
                             'sem': 'semestre',
                             'notanumericaacta': 'nota_numerica',
                             'notaalfaacta': 'nota_alfa'}, inplace=True)

In [64]:
#Leaving grades only of those students with preinscription information and that we can connect. 

notes_df = pd.merge(notes_df,matricules_df, on=['NIA','pla'], how='left')
notes_df['identification'].isnull().sum()
notes_df = notes_df.dropna(subset='identification') 
#we are not interested in grades from students that we do not have their preinscription information 

In [65]:
notes_df['codi_grau'] = notes_df.apply(assign_codi_grau, axis=1)

In [67]:
notes_df.columns

Index(['any_academic', 'assignatura', 'codi_assignatura', 'grup', 'NIA', 'pla',
       'tipus', 'curs', 'semestre', 'exhaurides', 'superada', 'nota_numerica',
       'nota_alfa', 'identification', 'curs_matricula', 'DNI', 'codi_grau'],
      dtype='object')

In [68]:
notes_df.sample()

,any_academic,assignatura,codi_assignatura,grup,NIA,pla,tipus,curs,semestre,exhaurides,superada,nota_numerica,nota_alfa,identification,curs_matricula,DNI,codi_grau
15015,2017,FETF,200151,701,1459372,877,O,2,2,3,S,3.2,SS,9308167A / 21025,2016.0,9308167A,21025


In [69]:
columns_to_drop = ['identification','curs_matricula','DNI']

notes_df = notes_df.drop(columns=columns_to_drop)

# Saving modified dfs

In [73]:
#Saving the clean dfs into the clean data folder
preinscripcions_df.to_csv('../data/processed/preinscripcions_df.csv', index=False)
students_df.to_csv('../data/processed/students_df.csv', index=False)
uni_access_df.to_csv('../data/processed/uni_access_df.csv', index=False)
previous_education_df.to_csv('../data/processed/previous_education_df.csv', index=False)
titulacions_df.to_csv('../data/processed/titulacions_df.csv', index=False )
matricules_df.to_csv('../data/processed/matricules_df.csv', index=False)
notes_df.to_csv('../data/processed/notes_df.csv', index=False)